Setup guide: Shift to readme

In [1]:
%%bash

cat <<EOL > setup_env.sh
source ~/env_MPI/bin/activate
export PYTHONPATH=~/env_MPI/lib/python3.11/site-packages:\$PYTHONPATH
module load python3/3.11.0
module load openmpi/4.0.5

export n_meshparts=24
export work_dir=~/PCG_MPI_Solver
cd \$work_dir
EOL


In [2]:
%%bash
source setup_env.sh

export model_name=Concrete
export scratch_path=$work_dir/$model_name
export input_model_path=$scratch_path/$model_name.zip

python3 read_input_model.py $work_dir $model_name $scratch_path $input_model_path


>creating directories..


>extracting files from Concrete


>success!


>elements:  124693


>nodes:     208316 


>dofs:      624948


>total runtime: 0.58 sec


In [3]:
%%bash
source setup_env.sh

python3 run_metis.py $n_meshparts



>loading model data..


>generating indices for 24 mesh parts..


>success!


>total runtime: 0.58 sec


In [4]:
%%bash
source setup_env.sh

export n_cores=4   
mpiexec -np $n_cores --map-by numa python3 partition_mesh.py $n_meshparts 0


>loading model data..


>partitioning mesh into 24 parts using 4 cores..


>success!


>total runtime: 3.36 sec


In [5]:
%%bash
source setup_env.sh

python3 -c "
from file_operations import exportz

TimeHistoryParam={'ExportFlag': True, # export results
                  'ExportFrmRate' : 1,
                  'ExportFrms' : [],
                  'PlotFlag' : False,
                  'TimeStepDelta': [0,1],
                  'ExportVars': 'U'} #D U PS PE GS GE

SolverParam={'Tol' : 1e-7,
             'MaxIter' : 10000}

GlobSettings = {'TimeHistoryParam':TimeHistoryParam,
                'SolverParam':SolverParam}
exportz('__pycache__/GlobSettings.zpkl', GlobSettings)
"

In [6]:
%%bash
source setup_env.sh

mpiexec -np $n_meshparts --map-by numa python3 pcg_solver.py 1 0


>loading partitioned data..


>running parallel pcg solver with 24 cores..


>exporting results..


>success!


>file read time:     0.2 sec 


>calculation time:   5.1 sec


>communication time: 1.0 sec


>---------------------------


>total runtime:      6.2 sec 


In [7]:
%%bash
source setup_env.sh

mpiexec -np 1 python3 export_vtk.py 1 "U" "Full"


>loading model and result data..


>exporting vtk files..


>success!


>total runtime: 1.53 sec
